In [30]:

import pandas as pd
import numpy as np
from numpy import random
from sklearn.model_selection import train_test_split
import ast
import os
from PIL import Image
import yaml

# 일부 에러 데이터 정규화

In [31]:
import pandas as pd

def modify_csv(input_file, output_file, column_name, new_value, first_zero_frame_hd):
    # CSV 파일 읽기
    df = pd.read_csv(input_file)

    # 조건 설정: 첫 번째 0이 나오기 전까지만
    row_condition = df.index < first_zero_frame_hd

    # 조건에 해당하는 값 수정
    df.loc[row_condition, column_name] = new_value

    # 수정된 데이터 저장
    df.to_csv(output_file, index=False)

# 함수 사용 예시
# modify_csv(
#     "C:\\Users\\dawoo\\desktop\\sf7\\coding\\pytorch\\project\\data\\data\\output\\frame\\frame_hd_data.csv", 
#     "./data/data/output/frame/frame_hd_data_modify.csv", 
#     "bboxes", 
#     "[]", 
#     first_zero_frame_hd
# )

# modify_csv(
#     "./data/data/output/frame/frame_sealf_data.csv", 
#     "./data/data/output/frame/frame_sealf_data_modify.csv", 
#     "bboxes", 
#     "[]", 
#     first_zero_frame_sealf
# )

# modify_csv(
#     "./data/data/output/frame/frame_seamf_data.csv", 
#     "./data/data/output/frame/frame_seamf_data_modify.csv", 
#     "bboxes", 
#     "[]", 
#     first_zero_frame_seamf
# )


# 데이터 전처리
- 양품 : 75 * 4 = 300
- 불량품 : 300 * 4 = 1200 랜덤 샘플링

In [32]:
from sklearn.model_selection import train_test_split
import pandas as pd

# CSV 파일 로드 함수
def load_data_with_pandas(file_path):
    data = pd.read_csv(file_path)
    return data

# 데이터 로드
data_bumper = load_data_with_pandas('./data/data/output/bumper/bumper_data.csv')
data_door_ed = load_data_with_pandas('./data/data/output/door/door_ed_data.csv')
data_door_scratch = load_data_with_pandas('./data/data/output/door/door_scratch_data.csv')
data_fender = load_data_with_pandas('./data/data/output/fender/fender_ed_data.csv')
data_frame_ed = load_data_with_pandas('./data/data/output/frame/frame_ed_data.csv')
data_frame_hd = load_data_with_pandas('./data/data/output/frame/frame_hd_data_modify.csv')
data_frame_sealf = load_data_with_pandas('./data/data/output/frame/frame_sealf_data_modify.csv')
data_frame_seamf = load_data_with_pandas('./data/data/output/frame/frame_seamf_data_modify.csv')


# 0이 처음 출력되는 index 값 추출
first_zero_bumper = data_bumper[data_bumper['quality'] == 0].index[0]

first_zero_door_ed = data_door_ed[data_door_ed['quality'] == 0].index[0]
first_zero_door_scratch = data_door_scratch[data_door_scratch['quality'] == 0].index[0]

first_zero_fender = data_fender[data_fender['quality'] == 0].index[0]

first_zero_frame_ed = data_frame_ed[data_frame_ed['quality'] == 0].index[0]
first_zero_frame_hd = data_frame_hd[data_frame_hd['quality'] == 0].index[0]
first_zero_frame_sealf = data_frame_sealf[data_frame_sealf['quality'] == 0].index[0]
first_zero_frame_seamf = data_frame_seamf[data_frame_seamf['quality'] == 0].index[0]



### index을 통해서 양품과 불량품의 위치를 구분

In [33]:
# print(f"bumper 양품 : {len(data_bumper.iloc[:first_zero_bumper])}")
# print(f"bumper 불량품 : {len(data_bumper.iloc[first_zero_bumper:])}")

# print(f"door_ed 양품 : {len(data_door_ed.iloc[:first_zero_door_ed])}")
# print(f"door_ed 불양품 : {len(data_door_ed.iloc[first_zero_door_ed:])}")
# print(f"door_scratch 양품 : {len(data_door_scratch.iloc[:first_zero_door_scratch])}")
# print(f"door_scratch 불양품 : {len(data_door_scratch.iloc[first_zero_door_scratch:])}")

# print(f"data_fender 양품 : {len(data_fender.iloc[:first_zero_fender])}")
# print(f"data_fender 불량품 : {len(data_fender.iloc[first_zero_fender:])}")

# print(f"data_frame_ed 양품 : {len(data_frame_ed.iloc[:first_zero_frame_ed])}")
# print(f"data_frame_ed 불양품 : {len(data_frame_ed.iloc[first_zero_frame_ed:])}")
# print(f"data_frame_hd 양품 : {len(data_frame_hd.iloc[:first_zero_frame_hd])}")
# print(f"data_frame_hd 불양품 : {len(data_frame_hd.iloc[first_zero_frame_hd:])}")
# print(f"data_frame_sealf 양품 : {len(data_frame_sealf.iloc[:first_zero_frame_sealf])}")
# print(f"data_frame_sealf 불양품 : {len(data_frame_sealf.iloc[first_zero_frame_sealf:])}")
# print(f"data_frame_seamf 양품 : {len(data_frame_seamf.iloc[:first_zero_frame_seamf])}")
# print(f"data_frame_seamf 불양품 : {len(data_frame_seamf.iloc[first_zero_frame_seamf:])}")



### 양품 데이터 추출

In [34]:
import pandas as pd
data_door = pd.concat([data_door_ed.iloc[:first_zero_door_ed],
                      data_door_scratch.iloc[:first_zero_door_scratch]], ignore_index=True)
data_frame = pd.concat([data_frame_ed.iloc[:first_zero_frame_ed],
                        data_frame_hd.iloc[:first_zero_frame_hd], data_frame_sealf.iloc[:first_zero_frame_sealf], data_frame_seamf.iloc[:first_zero_frame_seamf]], ignore_index=True)

bumper_before_zero = data_bumper.iloc[:first_zero_bumper]
fender_before_zero = data_fender.iloc[:first_zero_fender]


# 75개를 무작위로 추출 (복원 추출, random_state 설정)
bumper_ok_random_sample = bumper_before_zero.sample(
    n=min(75, len(bumper_before_zero)), replace=False, random_state=55)
fender_ok_random_sample = fender_before_zero.sample(
    n=min(75, len(fender_before_zero)), replace=False, random_state=55)
door_ok_random_sample = data_door.sample(n=75, replace=False)
frame_ok_random_sample = data_frame.sample(n=75, replace=False)
# 결과 출력
# print(bumper_ok_random_sample)
# print(len(bumper_ok_random_sample))

# print(fender_ok_random_sample)
# print(len(fender_ok_random_sample))

# print(door_ok_random_sample)
# print(len(door_ok_random_sample))

# print(frame_ok_random_sample)
# print(len(frame_ok_random_sample))

ok_data = pd.concat([bumper_ok_random_sample, fender_ok_random_sample,
                    door_ok_random_sample, frame_ok_random_sample], ignore_index=True, axis=0)

ok_data

,path,quality,part,bboxes
0,.\data\data\source_data\bumper\scratch\205_101...,1,범퍼,[]
1,.\data\data\source_data\bumper\scratch\205_101...,1,범퍼,[]
2,.\data\data\source_data\bumper\scratch\205_101...,1,범퍼,[]
3,.\data\data\source_data\bumper\scratch\205_101...,1,범퍼,[]
4,.\data\data\source_data\bumper\scratch\205_101...,1,범퍼,[]
...,...,...,...,...
295,.\data\data\source_data\frame/Hole_distortion\...,1,프레임,[]
296,.\data\data\source_data\frame\Seam_failure\207...,1,프레임,[]
297,.\data\data\source_data\frame/Hole_distortion\...,1,프레임,[]
298,.\data\data\source_data\frame/Hole_distortion\...,1,프레임,[]


### 불량품 데이터 추출

In [35]:
import pandas as pd
data_fault_bumper = data_bumper.iloc[first_zero_bumper:]
data_fault_door = pd.concat([data_door_ed.iloc[first_zero_door_ed:],
                             data_door_scratch.iloc[first_zero_door_scratch:]], ignore_index=True)
data_fault_fender = data_fender.iloc[first_zero_fender:]
data_fault_frame = pd.concat([data_frame_ed.iloc[first_zero_frame_ed:],
                              data_frame_hd.iloc[first_zero_frame_hd:], data_frame_sealf.iloc[first_zero_frame_sealf:], data_frame_seamf.iloc[first_zero_frame_seamf:]], ignore_index=True)


# 300개를 무작위로 추출 (복원 추출, random_state 설정)
bumper_fault_random_sample = data_fault_bumper.sample(n=300, replace=False)
fender_fault_random_sample = data_fault_fender.sample(n=300, replace=False)
door_fault_random_sample = data_fault_door.sample(n=300, replace=False)
frame_fault_random_sample = data_fault_frame.sample(n=300, replace=False)
# 결과 출력
# print(bumper_fault_random_sample)
# print(len(bumper_fault_random_sample))

# print(fender_fault_random_sample)
# print(len(fender_fault_random_sample))

# print(door_fault_random_sample)
# print(len(door_fault_random_sample))

# print(frame_fault_random_sample)
# print(len(frame_fault_random_sample))

fault_data = pd.concat([bumper_fault_random_sample, fender_fault_random_sample,
                        door_fault_random_sample, frame_fault_random_sample], ignore_index=True, axis=0)

fault_data

,path,quality,part,bboxes
0,.\data\data\source_data\bumper\scratch\205_101...,0,범퍼,"[[1622.093023255814, 1235.007536606374, 1764.6..."
1,.\data\data\source_data\bumper\scratch\205_101...,0,범퍼,"[[692.39879414298, 564.8924731182796, 1221.953..."
2,.\data\data\source_data\bumper\scratch\205_101...,0,범퍼,"[[1705.395348837209, 351.16564417177915, 511.0..."
3,.\data\data\source_data\bumper\scratch\205_101...,0,범퍼,"[[2446.6392291689176, 199.80734767025092, 256...."
4,.\data\data\source_data\bumper\scratch\205_101...,0,범퍼,"[[1046.5116279069766, 569.0322580645162, 1366...."
...,...,...,...,...
1195,.\data\data\source_data\frame\Seam_failure\207...,0,프레임,"[[1646.7906976744187, 434.25149700598797, 972...."
1196,.\data\data\source_data\frame\Seam_failure\207...,0,프레임,"[[1242.4186046511627, 650.2451612903225, 1506...."
1197,.\data\data\source_data\frame\Sealing_failure\...,0,프레임,"[[1664.3720930232557, 762.874251497006, 609.48..."
1198,.\data\data\source_data\frame\Exterior_damage\...,0,프레임,"[[1195.366824360471, 597.8312687687687, 978.54..."


In [36]:
# from sklearn.model_selection import train_test_split
# import pandas as pd



# # y에는 라벨, x에는 나머지 특성 정보
# y_ok = ok_data['quality']
# y_fault = fault_data['quality']

# # Stratified Split: 학습 데이터와 검증 데이터 비율을 80:20으로 설정, 클래스 비율을 유지
# train_data_ok, test_data_ok = train_test_split(y_ok, test_size=0.2, stratify=y_ok, random_state=60)
# train_data_fault, test_data_fault = train_test_split(y_fault, test_size=0.2, stratify=y_fault, random_state=60)

# # 학습 데이터에서 다시 80:20 비율로 나누어 검증 데이터 분리
# train_data_80_ok, valid_data_ok = train_test_split(train_data_ok, test_size=0.2, stratify=train_data_ok['quality'], random_state=60)
# train_data_80_fault, valid_data_fault = train_test_split(train_data_fault, test_size=0.2, stratify=train_data_fault['quality'], random_state=60)

# # CSV로 저장
# train_data_80_ok.to_csv('./data/train_data_80_ok.csv', index=False)
# train_data_80_fault.to_csv('./data/train_data_80_fault.csv', index=False)
# test_data_ok.to_csv('./data/test_data_ok.csv', index=False)
# test_data_fault.to_csv('./data/test_data_fault.csv', index=False)
# valid_data_ok.to_csv('./data/valid_data_ok.csv', index=False)
# valid_data_fault.to_csv('./data/valid_data_fault.csv', index=False)
# # 결과 출력
# print(f"y_ok 총 데이터 개수: {len(y_ok)}")
# print(f"y_fault 총 데이터 개수: {len(y_fault)}")
# print(f"train_data_80_ok 학습 데이터 개수: {len(train_data_80_ok)}")
# print(f"train_data_fault 학습 데이터 개수: {len(train_data_fault)}")
# print(f"test_data_ok 테스트 데이터 개수: {len(test_data_ok)}")
# print(f"test_data_fault 테스트 데이터 개수: {len(test_data_fault)}")
# print(f"valid_data_ok 검증 데이터 개수: {len(valid_data_ok)}")
# print(f"valid_data_fault 검증 데이터 개수: {len(valid_data_fault)}")


In [37]:
from sklearn.model_selection import train_test_split
import pandas as pd

# y에는 라벨, x에는 나머지 특성 정보
y_ok = ok_data['quality']
y_fault = fault_data['quality']

# Stratified Split: 학습 데이터와 검증 데이터 비율을 80:20으로 설정, 클래스 비율을 유지
train_data_ok, test_data_ok = train_test_split(
    ok_data, test_size=0.2, stratify=y_ok, random_state=60
)
train_data_fault, test_data_fault = train_test_split(
    fault_data, test_size=0.2, stratify=y_fault, random_state=60
)

# 학습 데이터에서 다시 80:20 비율로 나누어 검증 데이터 분리
train_data_80_ok, valid_data_ok = train_test_split(
    train_data_ok, test_size=0.2, stratify=train_data_ok['quality'], random_state=60)
train_data_80_fault, valid_data_fault = train_test_split(
    train_data_fault, test_size=0.2, stratify=train_data_fault['quality'], random_state=60)

# CSV로 저장
train_data_80_ok.to_csv('./data/train_data_80_ok.csv', index=False)
train_data_80_fault.to_csv('./data/train_data_80_fault.csv', index=False)
test_data_ok.to_csv('./data/test_data_ok.csv', index=False)
test_data_fault.to_csv('./data/test_data_fault.csv', index=False)
valid_data_ok.to_csv('./data/valid_data_ok.csv', index=False)
valid_data_fault.to_csv('./data/valid_data_fault.csv', index=False)

# 결과 출력
print(f"y_ok 총 데이터 개수: {len(y_ok)}")
print(f"y_fault 총 데이터 개수: {len(y_fault)}")
print(f"train_data_80_ok 학습 데이터 개수: {len(train_data_80_ok)}")
print(f"train_data_80_fault 학습 데이터 개수: {len(train_data_80_fault)}")
print(f"test_data_ok 테스트 데이터 개수: {len(test_data_ok)}")
print(f"test_data_fault 테스트 데이터 개수: {len(test_data_fault)}")
print(f"valid_data_ok 검증 데이터 개수: {len(valid_data_ok)}")
print(f"valid_data_fault 검증 데이터 개수: {len(valid_data_fault)}")


y_ok 총 데이터 개수: 300
y_fault 총 데이터 개수: 1200
train_data_80_ok 학습 데이터 개수: 192
train_data_80_fault 학습 데이터 개수: 768
test_data_ok 테스트 데이터 개수: 60
test_data_fault 테스트 데이터 개수: 240
valid_data_ok 검증 데이터 개수: 48
valid_data_fault 검증 데이터 개수: 192


# 테스트 데이터

In [38]:
# 이미지 복사
import shutil

def copy_images(data, output_dir):
    image_dir = os.path.join(output_dir, 'images')
    os.makedirs(image_dir, exist_ok=True)

    for index, row in data.iterrows():
        image_path = row['path']
        shutil.copy(image_path, image_dir)


# 학습 데이터와 검증 데이터를 각각 처리
copy_images(train_data_80_ok, './train')
copy_images(test_data_ok, './test')
copy_images(valid_data_ok,'./valid')
copy_images(train_data_80_fault, './train')
copy_images(test_data_fault, './test')
copy_images(valid_data_fault,'./valid')

### bboxes 데이터 추출

In [39]:
import ast

def parse_bboxes(data):
    """
    문자열 형태의 bounding box 정보를 파싱하여 리스트 형태로 변환하는 함수

    Args:
        data: 파싱할 데이터셋

    Returns:
        list: 파싱된 bounding box 리스트
    """

    bboxes_list = []
    for bbox_str in data["bboxes"]:
        try:
            bboxes = ast.literal_eval(bbox_str)
            bbox_list = []
            for bbox in bboxes:
                x, y, w, h = bbox
                bbox_list.append([x, y, w, h])
            bboxes_list.append(bbox_list)
        except (ValueError, SyntaxError) as e:
            print(f"Error parsing bbox: {e}")
    return bboxes_list

# 데이터셋별 bounding box 파싱
train_ok_bboxes = parse_bboxes(train_data_80_ok)
valid_ok_bboxes = parse_bboxes(valid_data_ok)
test_ok_bboxes = parse_bboxes(test_data_ok)
train_fault_bboxes = parse_bboxes(train_data_80_fault)
valid_fault_bboxes = parse_bboxes(valid_data_fault)
test_fault_bboxes = parse_bboxes(test_data_fault)

# 결과 출력 (예시)
print(train_ok_bboxes[:5])
print(valid_ok_bboxes[:5])
print(test_ok_bboxes[:5])
print(train_fault_bboxes[:5])
print(valid_fault_bboxes[:5])
print(test_fault_bboxes[:5])

[[], [], [], [], []]
[[], [], [], [], []]
[[], [], [], [], []]
[[[1529.5813953488373, 597.767441860465, 1119.348837209302, 240.2790697674419], [1324.4651162790697, 1523.7209302325582, 1248.2790697674423, 334.04651162790697], [1799.1627906976744, 1963.2558139534885, 1324.46511627907, 334.04651162790697]], [[1992.5581395348836, 187.78443113772454, 562.6046511627907, 1220.5988023952095]], [[1558.8837209302324, 645.5089820359282, 756.0, 604.431137724551]], [[997.6279069767443, 20.670731707317074, 1117.6744186046512, 1273.3170731707316], [865.1627906976744, 1099.6829268292681, 281.4883720930234, 231.51219512195144], [707.8604651162791, 698.6707317073171, 720.2790697674418, 351.4024390243901]], [[1277.5813953488373, 556.516129032258, 1265.860465116279, 603.3806451612903]]]
[[[1003.1162194540334, 925.953488372093, 1915.7857517950395, 492.2790697674419]], [[1695.8414284560722, 751.1377245508982, 431.50740875323004, 295.19012322577066]], [[1518.6815125849362, 720.0133384292913, 514.029452579194

# .yaml label 데이터 생성

### 1. 디렉토리 경로 전처리

In [40]:
import os
from PIL import Image


def extract_image_info(image_paths):
    """
    이미지 경로 리스트에서 이미지 크기, 디렉토리 경로, 파일 이름을 추출하는 함수

    Args:
        image_paths (list): 이미지 경로 리스트

    Returns:
        tuple: (image_sizes, image_dirs, file_names)
            image_sizes (list): 이미지 크기 리스트
            image_dirs (list): 이미지 디렉토리 경로 리스트
            file_names (list): 이미지 파일 이름 리스트
    """

    image_sizes = []
    image_dirs = []
    file_names = []

    for path in image_paths:
        try:
            image = Image.open(path)
            width, height = image.size
            image_sizes.append((width, height))
            image_dirs.append(os.path.dirname(path) + os.sep)
            file_names.append(os.path.splitext(os.path.basename(path))[0])
        except FileNotFoundError:
            print(f"파일을 찾을 수 없습니다: {path}")

    return image_sizes, image_dirs, file_names


# 데이터 준비 (예시)
train_ok_image_paths = train_data_80_ok.iloc[:, 0].tolist()
valid_ok_image_paths = valid_data_ok.iloc[:, 0].tolist()
test_ok_image_paths = test_data_ok.iloc[:, 0].tolist()
train_fault_image_paths = train_data_80_fault.iloc[:, 0].tolist()
valid_fault_image_paths = valid_data_fault.iloc[:, 0].tolist()
test_fault_image_paths = test_data_fault.iloc[:, 0].tolist()

# 이미지 정보 추출
train_sizes_ok, train_dirs_ok, train_names_ok = extract_image_info(train_ok_image_paths)
valid_sizes_ok, valid_dirs_ok, valid_names_ok = extract_image_info(valid_ok_image_paths)
test_sizes_ok, test_dirs_ok, test_names_ok = extract_image_info(test_ok_image_paths)
train_sizes_fault, train_dirs_fault, train_names_fault = extract_image_info(train_fault_image_paths)
valid_sizes_fault, valid_dirs_fault, valid_names_fault = extract_image_info(valid_fault_image_paths)
test_sizes_fault, test_dirs_fault, test_names_fault = extract_image_info(test_fault_image_paths)

# 결과 출력
# ok
print("train_sizes_ok 이미지 크기:", train_sizes_ok[:5])
print("train_dirs_ok 이미지 디렉토리:", train_dirs_ok[:5])
print("train_names_ok 파일 이름:", train_names_ok[:5])
print()
print("valid_sizes_ok 이미지 크기:", valid_sizes_ok[:5])
print("valid_dirs_ok 이미지 디렉토리:", valid_dirs_ok[:5])
print("valid_names_ok 파일 이름:", valid_names_ok[:5])
print()
print("test_sizes_ok 이미지 크기:", test_sizes_ok[:5])
print("test_dirs_ok 이미지 디렉토리:", test_dirs_ok[:5])
print("test_names_ok 파일 이름:", test_names_ok[:5])
# fault
print("train_sizes_fault 이미지 크기:", train_sizes_fault[:5])
print("train_dirs_fault 이미지 디렉토리:", train_dirs_fault[:5])
print("train_names_fault 파일 이름:", train_names_fault[:5])
print()
print("valid_sizes_fault 이미지 크기:", valid_sizes_fault[:5])
print("valid_dirs_fault 이미지 디렉토리:", valid_dirs_fault[:5])
print("valid_names_fault 파일 이름:", valid_names_fault[:5])
print()
print("test_sizes_fault 이미지 크기:", test_sizes_fault[:5])
print("test_dirs_fault 이미지 디렉토리:", test_dirs_fault[:5])
print("test_names_fault 파일 이름:", test_names_fault[:5])

c:\Users\dawoo\Desktop\SF7\coding\PYTORCH\pytorch-env\Lib\site-packages\PIL\Image.py:3186: DecompressionBombWarning: Image size (108000000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


train_sizes_ok 이미지 크기: [(4032, 3024), (4032, 1908), (4032, 1908), (2448, 2048), (4032, 1816)]
train_dirs_ok 이미지 디렉토리: ['.\\data\\data\\source_data\\bumper\\scratch\\', '.\\data\\data\\source_data\\bumper\\scratch\\', '.\\data\\data\\source_data\\fender\\Exterior_damage\\', '.\\data\\data\\source_data\\frame\\Seam_failure\\', '.\\data\\data\\source_data\\door\\scratch\\']
train_names_ok 파일 이름: ['205_101_10_1419f342-b811-454f-a371-57c02c6343c8', '205_101_10_60da5008-10b2-40b9-a0f6-e7dc22d2f11c', '209_102_10_319c5ffd-6e52-4711-a74f-bee9f0431a4f', '207_212_10_4455292f-a601-4515-a443-26a7474ebfc4', '204_101_10_8611f7b1-2614-4d1f-bed3-b32a22b96dc6']

valid_sizes_ok 이미지 크기: [(4032, 1960), (4032, 1908), (4000, 1868), (2448, 2048), (4032, 1908)]
valid_dirs_ok 이미지 디렉토리: ['.\\data\\data\\source_data\\fender\\Exterior_damage\\', '.\\data\\data\\source_data\\bumper\\scratch\\', '.\\data\\data\\source_data\\door\\scratch\\', '.\\data\\data\\source_data\\frame/Hole_distortion\\', '.\\data\\data\\sour

### 3. 부품 상태 전처리

In [51]:
# 부품 상태를 체크하는 함수
def get_label(part, status):
    if part == "범퍼":
        if status == 1:
            return 0
        elif status == 0:
            return 1
    elif part == "도어":
        if status == 1:
            return 0
        elif status == 0:
            return 2
    elif part == "휀더":
        if status == 1:
            return 0
        elif status == 0:
            return 3
    elif part == "프레임":
        if status == 1:
            return 0
        elif status == 0:
            return 4
    return None  # 기본적으로 반환할 값이 없으면 None

# 예시 부품과 상태
parts_and_status_train_ok = []
parts_and_status_valid_ok = []
parts_and_status_test_ok = []
for quality, part in zip(train_data_80_ok.iloc[:, 1], train_data_80_ok.iloc[:, 2]):
    parts_and_status_train_ok.append({"part": part, "status": quality})

for quality, part in zip(test_data_ok.iloc[:, 1], test_data_ok.iloc[:, 2]):
    parts_and_status_test_ok.append({"part": part, "status": quality})

for quality, part in zip(valid_data_ok.iloc[:, 1], valid_data_ok.iloc[:, 2]):
    parts_and_status_valid_ok.append({"part": part, "status": quality})

parts_and_status_train_fault = []
parts_and_status_valid_fault = []
parts_and_status_test_fault = []
for quality, part in zip(train_data_80_fault.iloc[:, 1], train_data_80_fault.iloc[:, 2]):
    parts_and_status_train_fault.append({"part": part, "status": quality})

for quality, part in zip(test_data_fault.iloc[:, 1], test_data_fault.iloc[:, 2]):
    parts_and_status_test_fault.append({"part": part, "status": quality})

for quality, part in zip(valid_data_fault.iloc[:, 1], valid_data_fault.iloc[:, 2]):
    parts_and_status_valid_fault.append({"part": part, "status": quality})

print(f"parts_and_status_train : {parts_and_status_train_ok}")
print(f"parts_and_status_test : {parts_and_status_test_ok}")
print(f"parts_and_status_valid : {parts_and_status_valid_ok}")

print(f"parts_and_status_train : {parts_and_status_train_fault}")
print(f"parts_and_status_test : {parts_and_status_test_fault}")
print(f"parts_and_status_valid : {parts_and_status_valid_fault}")

# 라벨링 결과를 저장할 리스트
labels_train_ok = []
labels_valid_ok = []
label_test_ok = []

# 각 부품 상태에 대해 라벨을 설정
for item in parts_and_status_train_ok:
    label = get_label(item["part"], item["status"])
    labels_train_ok.append(label)
for item in parts_and_status_test_ok:
    label = get_label(item["part"], item["status"])
    labels_valid_ok.append(label)
for item in parts_and_status_valid_ok:
    label = get_label(item["part"], item["status"])
    label_test_ok.append(label)

# # 결과 출력
print("labels_train_ok 라벨링 결과 :", labels_train_ok)
print("labels_valid_ok 라벨링 결과 :", labels_valid_ok)
print("label_test_ok 라벨링 결과 :", label_test_ok)

# 라벨링 결과를 저장할 리스트
labels_train_fault = []
labels_valid_fault = []
label_test_fualt = []

# 각 부품 상태에 대해 라벨을 설정
for item in parts_and_status_train_fault:
    label = get_label(item["part"], item["status"])
    labels_train_fault.append(label)
for item in parts_and_status_test_fault:
    label = get_label(item["part"], item["status"])
    labels_valid_fault.append(label)
for item in parts_and_status_valid_fault:
    label = get_label(item["part"], item["status"])
    label_test_fualt.append(label)

# # 결과 출력
print("labels_train_fault 라벨링 결과 :", labels_train_fault)
print("labels_valid_fault 라벨링 결과 :", labels_valid_fault)
print("label_test_fualt 라벨링 결과 :", label_test_fualt)

parts_and_status_train : [{'part': '범퍼', 'status': 1}, {'part': '범퍼', 'status': 1}, {'part': '휀더', 'status': 1}, {'part': '프레임', 'status': 1}, {'part': '도어', 'status': 1}, {'part': '범퍼', 'status': 1}, {'part': '도어', 'status': 1}, {'part': '휀더', 'status': 1}, {'part': '범퍼', 'status': 1}, {'part': '휀더', 'status': 1}, {'part': '프레임', 'status': 1}, {'part': '프레임', 'status': 1}, {'part': '범퍼', 'status': 1}, {'part': '도어', 'status': 1}, {'part': '휀더', 'status': 1}, {'part': '범퍼', 'status': 1}, {'part': '도어', 'status': 1}, {'part': '도어', 'status': 1}, {'part': '휀더', 'status': 1}, {'part': '범퍼', 'status': 1}, {'part': '프레임', 'status': 1}, {'part': '프레임', 'status': 1}, {'part': '범퍼', 'status': 1}, {'part': '도어', 'status': 1}, {'part': '휀더', 'status': 1}, {'part': '휀더', 'status': 1}, {'part': '프레임', 'status': 1}, {'part': '범퍼', 'status': 1}, {'part': '프레임', 'status': 1}, {'part': '휀더', 'status': 1}, {'part': '휀더', 'status': 1}, {'part': '범퍼', 'status': 1}, {'part': '프레임', 'status': 1}, {'part': 

In [52]:
def convert_to_yolo_format(bbox, label, image_width, image_height):
    yolo_labels = []  # 각 이미지에 대한 라벨을 담을 리스트

    if not bbox:
        # bbox가 비어있으면 기본값으로 라벨을 설정
        yolo_labels.append(f"{label} 0.0 0.0 0.0 0.0")
    else:
        # bbox가 있으면 여러 개의 파손 부위를 처리
        for x_min, y_min, width, height in bbox:
            # YOLO 형식에 맞게 좌표를 정규화
            x_center = (x_min + width / 2) / image_width
            y_center = (y_min + height / 2) / image_height
            norm_width = width / image_width
            norm_height = height / image_height
            # 정규화된 좌표와 함께 라벨을 저장
            yolo_labels.append(f"{label} {x_center:.6f} {y_center:.6f} {
                               norm_width:.6f} {norm_height:.6f}")

    return yolo_labels


def save_yolo_labels(output_dir, bboxes, labels, size, names):
    # 각 이미지에 대해 라벨을 텍스트 파일로 저장
    for bbox, label, (w, h), name in zip(bboxes, labels, size, names):
        yolo_labels = convert_to_yolo_format(bbox, label, w, h)

        # 라벨을 .txt 파일로 저장
        txt_file_path = os.path.join(output_dir, f"{name}.txt")
        with open(txt_file_path, 'w') as txt_file:
            for yolo_data in yolo_labels:
                txt_file.write(f"{yolo_data}\n")



# 라벨을 저장할 디렉토리
train_output_dir = './train/labels'
os.makedirs(train_output_dir, exist_ok=True)

# YOLO 라벨 저장
save_yolo_labels(train_output_dir, train_ok_bboxes, labels_train_ok, train_sizes_ok, train_names_ok)
save_yolo_labels(train_output_dir, train_fault_bboxes, labels_train_fault, train_sizes_fault, train_names_fault)
# 라벨을 저장할 디렉토리
valid_output_dir = './valid/labels'
os.makedirs(valid_output_dir, exist_ok=True)

# YOLO 라벨 저장
save_yolo_labels(valid_output_dir, valid_ok_bboxes, labels_valid_ok, valid_sizes_ok, valid_names_ok)
save_yolo_labels(valid_output_dir, valid_fault_bboxes, labels_valid_fault, valid_sizes_fault, valid_names_fault)

# 라벨을 저장할 디렉토리
test_output_dir = './test/labels'
os.makedirs(test_output_dir, exist_ok=True)

# YOLO 라벨 저장
save_yolo_labels(test_output_dir, test_ok_bboxes, label_test_ok, test_sizes_ok, test_names_ok)
save_yolo_labels(test_output_dir, test_fault_bboxes, label_test_fualt, test_sizes_fault, test_names_fault)

In [43]:
# YAML 파일에 저장
yaml_data = {
    "train": "C:/Users/dawoo/desktop/sf7/coding/pytorch/project/train/images",
    "test": "C:/Users/dawoo/desktop/sf7/coding/pytorch/project/test/images",
    "val": "C:/Users/dawoo/desktop/sf7/coding/pytorch/project/valid/images",
    "nc": 5,
    "names": ["ok",
              "bumper_faulty",
              "door_faulty",
              "fender_faulty",
              "frame_faulty"]
}

with open(f"data.yaml", "w") as yaml_file:
    yaml.dump(yaml_data, yaml_file, default_flow_style=False, sort_keys=False)

# print("YAML 파일이 생성되었습니다.")

## 모델 코드 작성

In [44]:
# from ultralytics import YOLO

# # YOLO 모델 객체 생성
# model = YOLO("yolov8s.pt")  # 사전 학습된 YOLOv8 모델 로드

# # 모델 학습 시작
# results = model.train(
#     data="data.yaml",       # YAML 파일 경로
#     epochs=20,              # 학습 에포크 수
#     imgsz=640,              # 입력 이미지 크기 (기본값: 640)
#     batch=16,               # 배치 크기
#     project="C:\\Users\\dawoo\\Desktop\\SF7\\coding\\PYTORCH\\project\\YOLO", # 학습 결과를 저장할 프로젝트 이름
#     name="part_damage_detection",   # 저장될 실행 이름
#     workers=4,              # 데이터 로드 병렬 처리 수
#     device=0                # GPU ID (0은 첫 번째 GPU를 의미, -1은 CPU)
# )


In [45]:
# import os
# from ultralytics import YOLO

# # 데이터 설정 파일 경로
# data_yaml = "data.yaml"

# # YOLO 모델 정의 (랜덤 초기화로 학습 시작)
# model = YOLO("yolov8s.pt")
# # model.model = model.model.reset_parameters()  # 가중치 초기화 (랜덤 가중치)

# # 학습 설정
# model.train(
#     data=data_yaml,          # 데이터 설정 파일 경로
#     epochs=50,               # 학습 에폭 수
#     batch=16,                # 배치 크기
#     imgsz=640,               # 입력 이미지 크기
#     device=0,                # GPU 사용
#     workers=4,               # 데이터 로드 병렬 처리 워커 수
#     project="C:\\Users\\dawoo\\Desktop\\SF7\\coding\\PYTORCH\\project\\YOLO",     # 결과 저장 디렉토리
#     name="part_damage_detection",       # 실험 이름
#     conf=0.5,
#     lr0=0.001,  # 초기 학습률
#     optimizer='AdamW',  # 옵티마이저 설정 (SGD, Adam, AdamW 등)
#     weight_decay=0.01   # 가중치 감소 설정
# )
# model.save('part_damage_detection.pt')

In [46]:
# import os
# from ultralytics import YOLO

# # 데이터 설정 파일 경로
# data_yaml = "data.yaml"

# # YOLO 모델 정의 (yolov8m.pt 사용)
# model = YOLO("yolov8n.pt")

# # 학습 설정
# model.train(
#     data=data_yaml,
#     epochs=70,
#     batch=8,  # 배치 크기 감소
#     imgsz=640,  # 이미지 크기
#     device=0,
#     workers=3,
#     project="YOLO1",
#     name="part_damage_detection",
#     conf=0.5,
#     lr0=0.0001,
#     optimizer='AdamW',
#     weight_decay=0.01,
#     patience=10,
# )

# model.save('part_damage_detection2.pt')

In [47]:
# import os
# from ultralytics import YOLO

# # 데이터 설정 파일 경로
# data_yaml = "data.yaml"

# # YOLO 모델 정의 (yolov8m.pt 사용)
# model = YOLO("yolov8s.pt")

# # 학습 설정
# model.train(
#     data=data_yaml,
#     epochs=50,
#     batch=8,  # 배치 크기 감소
#     imgsz=640,  # 이미지 크기 줄이기
#     device=0,
#     workers=4,
#     project="YOLO2",
#     name="part_damage_detection",
#     conf=0.5,
#     lr0=0.0005,
#     optimizer='AdamW',
#     weight_decay=0.005,
#     patience=10,
# )

# model.save('part_damage_detection3.pt')

In [48]:
# # 모델 평가
# metrics = model.val()

In [49]:
# # 모델 로드 및 추론
# model = YOLO("C:\\Users\\dawoo\\desktop\\sf7\\coding\\pytorch\\project\\model_8_s_50_16_944.pt")
# results = model.predict(source="C:\\Users\\dawoo\\desktop\\sf7\\coding\\pytorch\\project\\test\\images", save=True, imgsz=640)



In [50]:
count = 0
for result in results:
    # 바운딩 박스가 없는 경우 'no detections'으로 간주
    if result.boxes is None or len(result.boxes.xywh) == 0:
        count += 1

print(f"no detections : {count}")
print(f"len(results) : {len(results)}")


NameError: name 'results' is not defined

In [ ]:
# import cv2
# from ultralytics import YOLO

# # YOLOv8 모델 로드
# model_path = "C:\\Users\\dawoo\\desktop\\sf7\\coding\\pytorch\\project\\model_8_s_50_16_944.pt"  # 학습한 모델 경로
# model = YOLO(model_path)
# model.to("cuda")

# # 카메라 열기
# cap = cv2.VideoCapture(0)  # 기본 카메라 (ID 0)
# if not cap.isOpened():
#     print("카메라를 열 수 없습니다.")
#     exit()

# # 카메라 프레임 크기 설정 (선택 사항)
# cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
# cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# # 색상 설정 (바운딩 박스와 텍스트)
# box_color = (0, 255, 0)  # 녹색
# text_color = (255, 255, 255)  # 흰색

# print("Press 'q' to exit...")

# while True:
#     # 프레임 읽기
#     ret, frame = cap.read()
#     if not ret:
#         print("프레임을 읽을 수 없습니다.")
#         break
#     flip_frame = cv2.flip(frame, 1)
#     # 모델 추론 (YOLOv8)
#     results = model(source=frame, conf=0.5, show=False,
#                     verbose=False)  # 이미지에 대해 추론
#     detections = results[0]  # 첫 번째 결과 가져오기

#     # 바운딩 박스와 라벨 그리기
#     for box in detections.boxes:
#         # 바운딩 박스 좌표 (정규화된 값 아님)
#         x1, y1, x2, y2 = map(int, box.xyxy[0])  # 좌상단(x1, y1), 우하단(x2, y2)
#         conf = box.conf[0]  # 신뢰도
#         cls = int(box.cls[0])  # 클래스 ID
#         label = f"{model.names[cls]} {conf:.2f}"  # 클래스 이름과 신뢰도

#         # 바운딩 박스와 라벨 추가
#         cv2.rectangle(frame, (x1, y1), (x2, y2), box_color, 2)  # 바운딩 박스
#         cv2.putText(frame, label, (x1, y1 - 10),
#                     cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 2)

#     # 프레임 화면에 표시
#     cv2.imshow("YOLOv8 Real-Time Detection", flip_frame)

#     # 'q' 키를 누르면 종료
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# # 자원 해제
# cap.release()
# cv2.destroyAllWindows()

## 손실 그래프

In [ ]:
# loss function
import matplotlib.pyplot as plt

# 손실 감소 그래프
plt.subplot(1, 2, 1)
plt.plot(losses)
plt.grid()
plt.title('Loss over Epochs')
plt.xlabel('epoch')
plt.ylabel('Loss')

## 데이터 검증

In [ ]:
## data evaluation

from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_curve, roc_auc_score
pred=model(y_test)


# 혼동 행렬
confmat = confusion_matrix(y_true=y_test, y_pred=pred)
print('혼동 행렬: ', confmat)


# 정확도
accuracy=accuracy_score(y_true=y_test, y_pred=pred)
print('정확도: ', accuracy)


# 정밀도(precision)
precision = precision_score(y_true=y_test, y_pred=pred)
print('정밀도: ', precision)


# 재현율(recall)
recall = recall_score(y_true=y_test, y_pred=pred)
print('재현율: ', recall)


# roc auc
fpr, tpr, _ = roc_curve(y_true=y_test, y_pred=pred)

plt.plot(fpr, tpr)
plt.xlabel('FP Rate')
plt.ylabel('TP Rate')

plt.show()

print('auc 점수: ', roc_auc_score(y_test, pred))